In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from nn_builder.pytorch.NN import NN

# 读取数据
df = pd.read_csv('output.csv')

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# 初始化scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 将数据分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.99, random_state=42)

# 转换为Tensor
train_features = torch.tensor(X_train, dtype=torch.float32)
train_targets = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
test_features = torch.tensor(X_test, dtype=torch.float32)
test_targets = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# 检查是否有可用的GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 将数据移到GPU
train_features = train_features.to(device)
train_targets = train_targets.to(device)
test_features = test_features.to(device)
test_targets = test_targets.to(device)

# 创建DataLoader
train_dataset = TensorDataset(train_features, train_targets)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 初始化模型、损失函数和优化器
model = NN(input_dim=16, layers_info=[150, 100, 50, 50, 50, 5, 1],
           dropout=0.5, initialiser='xavier', batch_norm=True).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 训练循环
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 保存训练的模型和scaler
checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item(),
    'scaler': scaler
}
torch.save(checkpoint, 'model_checkpoint.pth')

# 评估模型
model.eval()
# with torch.no_grad():
#     test_outputs = model(test_features)

# test_outputs = test_outputs.cpu().numpy()
# test_targets = test_targets.cpu().numpy()

# # 打印十个样本的实际值和预测值
# for i in range(10):
#     print(f"Sample {i+1}:")
#     print(f"Actual Value: {test_targets[i]}")
#     print(f"Predicted Value: {test_outputs[i][0]}")
#     print("-" * 30)


layer info  [150, 100, 50, 50, 50, 5, 1]
layer info  [150, 100, 50, 50, 50, 5, 1]
Epoch [1/200], Loss: 26.5552
Epoch [11/200], Loss: 1.0989
Epoch [21/200], Loss: 3.4630
Epoch [31/200], Loss: 0.7195
Epoch [41/200], Loss: 4.4553
Epoch [51/200], Loss: 3.0923
Epoch [61/200], Loss: 1.7093
Epoch [71/200], Loss: 2.2547
Epoch [81/200], Loss: 2.7175
Epoch [91/200], Loss: 1.1739
Epoch [101/200], Loss: 7.3690
Epoch [111/200], Loss: 3.3219
Epoch [121/200], Loss: 3.7217
Epoch [131/200], Loss: 0.7455
Epoch [141/200], Loss: 0.8475
Epoch [151/200], Loss: 2.3984
Epoch [161/200], Loss: 1.2336
Epoch [171/200], Loss: 6.2911
Epoch [181/200], Loss: 0.8235
Epoch [191/200], Loss: 8.0894


NN(
  (embedding_layers): ModuleList()
  (hidden_layers): ModuleList(
    (0): Linear(in_features=16, out_features=150, bias=True)
    (1): Linear(in_features=150, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3-4): 2 x Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=5, bias=True)
  )
  (output_layers): ModuleList(
    (0): Linear(in_features=5, out_features=1, bias=True)
  )
  (dropout_layer): Dropout(p=0.5, inplace=False)
  (batch_norm_layers): ModuleList(
    (0): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2-4): 3 x BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [11]:
import pygame
import numpy as np
import sympy as sp
import random
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

pygame.init()

BOARD_SIZE_X = 5
BOARD_SIZE_Y = 10
TILE_SIZE_X = 200
TILE_SIZE_Y = 60
TOTAL_COUNT = 0
GOAL_COUNT = 0

WIDTH, HEIGHT = BOARD_SIZE_X * TILE_SIZE_X, BOARD_SIZE_Y * TILE_SIZE_Y
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("DeepBalls")

# Colors
GREEN_ONE = (8, 102, 38)
GREEN_TWO = (0, 255, 82)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
font = pygame.font.Font(None, 48)

board = np.zeros((BOARD_SIZE_X, BOARD_SIZE_Y), dtype=int)
output_text = font.render("0", True, (255, 255, 255))

if_goal = False

running = True

# Arrow configurations
arrow_offsets = {
    BLUE: [(1.7, 0), (1, 1), (1, -1)],
    RED: [(-1.7, 0), (-1, 1), (-1, -1)]
}

new_directions = {
    BLUE: [(1, 0), (1, 1), (1, -1)],
    RED: [(-1, 0), (-1, 1), (-1, -1)]
}

arrows = []


class Player:
    def __init__(self, x, y, color):
        self.position = (x * TILE_SIZE_X, y * TILE_SIZE_Y)
        self.color = color


# 定义黑色球员和红色球员的位置
black_players = [
    Player(3, 5, BLUE),
    Player(1, 8, BLUE),
    Player(1, 2, BLUE)
]

red_players = [
    Player(2, 5, RED),
    Player(4, 2, RED),
    Player(4, 8, RED)
]

# 合并球员列表
players = black_players + red_players

columns = ['Ball Position X', 'Ball Position Y', 'Ball Direction X', 'Ball Direction Y']
for i in range(len(players)):
    columns.extend([f'Player {i} Position X', f'Player {i} Position Y'])
columns.append('Steps to Goal')
    
df_temp = pd.DataFrame(columns=columns)

class Ball:
    def __init__(self, x, y, direction):
        self.position = (x * TILE_SIZE_X, y * TILE_SIZE_Y)
        self.direction = direction


# 定义球的位置和方向
ball = Ball(3, 5, (1, 1))  # 球的位置在 (3, 5)，方向为向右


# 绘制棋盘
def draw_board():
    for row in range(BOARD_SIZE_Y):
        for col in range(BOARD_SIZE_X):
            color = BLACK
            pygame.draw.rect(screen, color,
                             pygame.Rect(col * TILE_SIZE_X, row * TILE_SIZE_Y, TILE_SIZE_X - 2, TILE_SIZE_Y - 2))


# 绘制球员
def draw_players():
    for player in players:
        col, row = player.position
        pygame.draw.circle(screen, player.color,
                           (col, row),
                           TILE_SIZE_Y // 4)


# 绘制球和箭头
def draw_ball_and_arrow():
    col, row = ball.position
    pygame.draw.circle(screen, WHITE,
                       (int(col), int(row)),
                       TILE_SIZE_Y // 4)

    # 绘制箭头
    arrow_start = (int(col), int(row))
    arrow_end = (int(arrow_start[0] + ball.direction[0] * 20),
                 int(arrow_start[1] + ball.direction[1] * 20))
    pygame.draw.line(screen, WHITE, arrow_start, arrow_end, 5)
    pygame.draw.polygon(screen, WHITE, [
        (int(arrow_end[0] + ball.direction[0] * 10), int(arrow_end[1] + ball.direction[1] * 10)),
        (int(arrow_end[0] - ball.direction[1] * 10), int(arrow_end[1] + ball.direction[0] * 10)),
        (int(arrow_end[0] + ball.direction[1] * 10), int(arrow_end[1] - ball.direction[0] * 10))
    ])

    for arrow in arrows:
        draw_arrow(arrow['start'], arrow['end'], arrow['color'])


def draw_arrow(start, end, color):
    pygame.draw.polygon(screen, color, [
        (int(end[0] + (end[0] - start[0]) * 0.2), int(end[1] + (end[1] - start[1]) * 0.2)),
        (int(end[0] - (end[1] - start[1]) * 0.2), int(end[1] + (end[0] - start[0]) * 0.2)),
        (int(end[0] + (end[1] - start[1]) * 0.2), int(end[1] - (end[0] - start[0]) * 0.2))
    ])


def show_arrows(player):
    global arrows
    x, y = player.position
    arrows = []
    for offset in arrow_offsets[player.color]:
        end_x = x + offset[0] * 20
        end_y = y + offset[1] * 20
        arrows.append({
            'start': (x, y),
            'end': (end_x, end_y),
            'color': player.color,
            'direction': offset
        })


# 处理球的运动逻辑
def move_ball():
    global arrows, GOAL_COUNT, if_goal
    x, y = sp.symbols('x y')
    col, row = ball.position
    direction = ball.direction

    # 定义球的运动方程
    line_eq = sp.Eq((y - row) * direction[0], (x - col) * direction[1])

    # print(ball.position, ball.direction)

    # 寻找最近的交点
    intersection_points = []
    for i in range(BOARD_SIZE_X + 1):
        if direction[0] != 0:
            x_val = i * TILE_SIZE_X
            if ball.direction[0] * (x_val - col) > 0:
                y_val = sp.solve(line_eq.subs(x, x_val), y)[0]
                formatted_tuple = tuple(format(x, '.20f') for x in ball.position)
                if 0 <= y_val <= HEIGHT:
                    intersection_points.append((x_val, y_val))

    for j in {0, 10}:
        if direction[1] != 0:
            y_val = j * TILE_SIZE_Y
            if ball.direction[1] * (y_val - row) > 0:
                x_val = sp.solve(line_eq.subs(y, y_val), x)[0]
                if 0 <= x_val <= WIDTH:
                    intersection_points.append((x_val, y_val))

    # 找到最近的交点
    next_position = min(intersection_points, key=lambda p: ((p[0] - col) ** 2 + (p[1] - row) ** 2) ** 0.5)

    if_goal = False

    # 检查碰撞边框
    if int(next_position[0]) == 0 or int(next_position[0]) == WIDTH:
        direction = (-direction[0], direction[1])
        if 240 <= next_position[1] <= 360:
            GOAL_COUNT += 1
            if_goal = True
    if int(next_position[1]) == 0 or int(next_position[1]) == HEIGHT:
        direction = (direction[0], -direction[1])

    ball.position = next_position
    ball.direction = direction

    for p in players:  # to do 边界时随机可以改成二选一, 应该拆除去
        new_y = p.position[1] + random.choice([-TILE_SIZE_Y, 0, TILE_SIZE_Y])
        if 0 <= new_y <= HEIGHT:
            p.position = (p.position[0], new_y)


def move_players():
    for p in players:
        if ((p.position[0] - ball.position[0]) ** 2 + (p.position[1] - ball.position[1]) ** 2) ** 0.5 < 40:
            # print("shoot!!!!!")
            show_arrows(p)
            ball.direction = random.choice(new_directions[p.color])
            break
        
def grade_state():
    global df_temp, output_text, font
    row = [
        ball.position[0], ball.position[1], ball.direction[0], ball.direction[1]
    ]
    for player in players:
        row.extend([player.position[0], player.position[1]])

    row = np.array(row).reshape(1, -1)
    row = scaler.transform(row)
    print(row)
    feature_tensor = torch.tensor(row, dtype=torch.float32).to(device)
    with torch.no_grad():
        output = model(feature_tensor).item()
    output_text = font.render(str(output), True, (255, 255, 255))
    print(output)

# 处理鼠标点击事件
def handle_click(pos):
    global ball, output_text
    x, y = pos

    arrows.clear()
    move_ball()
    move_players()
    grade_state()


while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            TOTAL_COUNT += 1
            handle_click(pygame.mouse.get_pos())

    screen.fill(GREEN_ONE)
    draw_board()
    draw_players()
    draw_ball_and_arrow()
    screen.blit(output_text, (10, 10))
    pygame.display.flip()

pygame.quit()


[[ 0.75242128  1.02589656  0.59190279  1.18047276  0.         -0.00317611
   0.          0.93446096  0.         -0.99127051  0.          0.29369377
   0.         -1.24874468  0.          1.24850853]]
1.5578663349151611
[[ 1.11830249  1.53961086  0.59190279 -1.18151458  0.         -0.32144087
   0.          0.93446096  0.         -1.31114587  0.         -0.02059892
   0.         -0.93686725  0.          1.24850853]]
1.3023524284362793
[[ 1.48418369  1.02589656 -1.68946662 -1.18151458  0.         -0.00317611
   0.          1.25493688  0.         -0.99127051  0.         -0.33489161
   0.         -0.93686725  0.          0.93612069]]
0.9545469284057617
[[ 7.52421284e-01 -1.53205359e-03 -1.68946662e+00 -1.18151458e+00
   0.00000000e+00  3.15088658e-01  0.00000000e+00  9.34460956e-01
   0.00000000e+00 -6.71395152e-01  0.00000000e+00 -3.34891613e-01
   0.00000000e+00 -9.36867249e-01  0.00000000e+00  1.24850853e+00]]
6.435876846313477
[[ 0.02065887 -1.02896066 -1.68946662 -1.18151458  0.      